In [2]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
import glob
import os
from IPython.display import display
import matplotlib.pyplot as plt
import json
from sklearn import preprocessing
from keras.models import Model
from keras.layers import Input, LSTM, Dense
from keras.preprocessing.text import Tokenizer
import tensorflow_addons as tfa
from sklearn import preprocessing

In [3]:
film_csv = "/Volumes/Seagate/natasha-diploma/content_data.csv"
trailers_csv = "/Volumes/Seagate/natasha-diploma/trailers.csv"
sessions_csv = "/Volumes/Seagate/natasha-diploma/sessions.csv"

films = pd.read_csv(film_csv, index_col=None, header=0)
display(films.head())

trailers_meta = pd.read_csv(trailers_csv, index_col=None, header=0)
display(trailers_meta.head())



sessions = pd.read_csv(sessions_csv, index_col=None, header=0)
sessions.drop(sessions.columns[sessions.columns.str.contains('Unnamed',case = False)],axis = 1, inplace = True)
        
display(sessions.head())

sess = sessions.to_numpy().tolist()

in_sess = []
out_sess = []
for i in range(len(sess)):
    in_sess.append(sess[i][:5])
    out_sess.append(sess[i][5:])

,id,title,duration,release_date,restrict,orig_title,kinopoisk_id,imdb_rating,kinopoisk_rating,synopsis,categories,genres,properties
0,60432,Философия Фила,5816.0,2019-04-05,18.0,Phil,960568,5.4,5.86,"Фил – стоматолог, страдающий от депрессии. Его...",{Фильмы},"{Драмы,Зарубежные,Комедии}","{ ""Место действия"" : [""США""], ""Качества"" : [""д..."
1,64219,Зубная фея,5123.0,2019-04-02,18.0,Tooth Fairy,1126022,2.2,0.00,Старая женщина начинает рассказывать легенду о...,{Фильмы},"{Зарубежные,Триллеры,Ужасы}","{ ""Место действия"" : [""не определено""], ""Качес..."
2,65407,Проснись,4667.0,2019-01-19,16.0,Wake Up,1069713,2.9,0.00,"Врач-психиатр исследует дневник девушки, котор...",{Фильмы},"{Боевики,Зарубежные,Криминал,Триллеры,Ужасы}","{ ""Аудитория"" : [""для взрослых"",""для подготовл..."
3,75149,[4k] Адская кухня,6148.0,2019-08-09,16.0,The Kitchen,1117988,5.5,5.50,"Смотрите в 4К! Три жены ирландских гангстеров,...",{Фильмы},"{Боевики,Драмы,Зарубежные,Криминал,По комиксам}","{ ""Настроение"" : [""захватывающие"",""авантюрный""..."
4,75629,[4k] Покемон. Детектив Пикачу,6013.0,2019-05-02,12.0,Pokémon Detective Pikachu,994864,6.6,6.56,Смотрите в 4K! Элитный детектив Пикачу отправл...,"{Для детей,Фильмы}","{Детективы,Зарубежные,Комедии,Семейные,Фильмы,...","{ ""Аудитория"" : [""для детей 13-16 лет"",""9 лет""..."


,Unnamed: 0,Unnamed: 0.1,title_x,duration,release_date,restrict,orig_title,kinopoisk_id,imdb_rating,kinopoisk_rating,...,emotions,Place of act,Qualities,Time of act,Based on,Audience,Mood,Subgenre,About,Theme
0,0,0,Философия Фила,5816.0,2019-04-05,18.0,Phil,960568,5.4,5.86,...,"{'neutral': 0.5312193632125854, 'negative': 0....",['США'],['дебютный'],"['21-й век', 'новейшее время']",['оригинального сценария'],"['для молодёжи', 'для взрослых']","['грустный', 'депрессивный', 'трогательный']","['Драмы', 'Комедийные']","['преследования', 'самозванцы', 'расследование...","['развлекательные', 'психологический']"
1,1,4,Зубная фея,5123.0,2019-04-02,18.0,Tooth Fairy,1126022,2.2,0.00,...,"{'negative': 0.4688006341457367, 'skip': 0.392...",['не определено'],"['зрелищные', 'жанровый', 'неожиданный финал']",['не определено'],"['эпоса и легенд', 'оригинального сценария']","['для взрослых', 'для мужчин', 'для подготовле...","['трагичные', 'мрачные', 'шокирующие', 'захват...","['ужасы', 'триллер']","['преследования', 'страхи', 'дети', 'молодежь'...",['развлекательные']
2,2,5,Проснись,4667.0,2019-01-19,16.0,Wake Up,1069713,2.9,0.00,...,"{'positive': 0.3140605390071869, 'negative': 0...",['не определено'],"['жанровый', 'дебютный']",['новейшее время'],['оригинального сценария'],"['для взрослых', 'для подготовленного зрителя'...","['захватывающие', 'жестокие', 'напряженные', '...","['Драмы', 'Боевики', 'триллер']","['психотерапевты', 'кровь', 'тайна смерти', 'п...","['психологический', 'развлекательные']"
3,3,6,[4k] Адская кухня,6148.0,2019-08-09,16.0,The Kitchen,1117988,5.5,5.50,...,"{'neutral': 0.4765896201133728, 'negative': 0....","['большой город', 'США']","['жанровый', 'остросюжетные']",['20-й век'],['комиксов'],"['для мужчин', 'для женщин', 'для взрослых']","['захватывающие', 'авантюрный', 'жестокие', 'н...","['Боевики', 'Драмы', 'криминальные']","['враги или вражда', 'женщины', 'антигерои', '...","['авантюрные', 'гангстерская']"
4,4,7,[4k] Покемон. Детектив Пикачу,6013.0,2019-05-02,12.0,Pokémon Detective Pikachu,994864,6.6,6.56,...,"{'neutral': 0.4532718360424042, 'negative': 0....","['вымышленная страна (город)', 'большой город']","['частично анимационный', 'зрелищные', 'кассов...",['21-й век'],"['оригинального сценария', 'сериалов']","['для детей 13-16 лет', '9 лет', '8 лет', '7-1...","['авантюрный', 'захватывающие', 'динамичный', ...","['детские', 'Приключенческие', 'Фантастические...","['притяжение противоположностей', 'риск', 'пре...","['развлекательные', 'образовательный']"


,0,1,2,3,4,5,6,7,8,9
0,60432,174457,184174,111847,172257,227812,250885,174558,212559,214967
1,65407,225043,138191,177902,176620,225371,190349,147590,181198,138245
2,75149,60432,133587,148584,186787,222725,212087,174369,423155,431690
3,75629,205372,186058,171392,160160,184174,172257,75630,226747,212559
4,75630,172257,219611,211554,248722,250977,216926,190967,415428,419254


In [4]:
def flatten(l): return flatten(l[0]) + (flatten(l[1:]) if len(l) > 1 else []) if type(l) is list else [l]

def vocab_creater(arr, add_num):
    json_arr = []
    for i in range(len(arr)):
        js = json.loads(arr[i].replace('\'', '\"'))
        json_arr.append(js)
    flat_arr = flatten(json_arr)
    dic = list(dict.fromkeys(flat_arr))
    word2idx = {}
    idx2word = {}
    for i in range(len(dic)):
        word2idx[dic[i]] = i + add_num
        idx2word[i + add_num] = dic[i]

    return word2idx, idx2word, len(dic)

In [5]:
trailers_meta_array = []

time_of_act = trailers_meta['Time of act'].values
place_of_act = trailers_meta['Place of act'].to_numpy()
qualities = trailers_meta['Qualities'].to_numpy()
based_on = trailers_meta['Based on'].to_numpy()
content_id = trailers_meta['content_id'].values
audience = trailers_meta['Audience'].to_numpy()
mood = trailers_meta['Mood'].to_numpy()
subgenre = trailers_meta['Subgenre'].to_numpy()
about = trailers_meta['About'].to_numpy()
theme = trailers_meta['Theme'].to_numpy()

add_num = 0
time_vocab = vocab_creater(time_of_act, add_num)
add_num += time_vocab[2] + 10
place_vocab = vocab_creater(place_of_act, add_num)
add_num += place_vocab[2]
qua_vocab = vocab_creater(qualities, add_num)
add_num += qua_vocab[2]
based_vocab = vocab_creater(based_on, add_num)
add_num += based_vocab[2]
audi_vocab = vocab_creater(audience, add_num)
add_num += audi_vocab[2]
mood_vocab = vocab_creater(mood, add_num)
add_num += mood_vocab[2]
subgenr_vocab = vocab_creater(subgenre, add_num)
add_num += subgenr_vocab[2]
about_vocab = vocab_creater(about, add_num)
add_num += about_vocab[2]
theme_vocab = vocab_creater(theme, add_num)
    

In [6]:
def tokenize_arr(arr, vocab):
    json_arr = []
    for i in range(len(arr)):
        js = json.loads(arr[i].replace('\'', '\"'))
        json_arr.append(js)
    vocabs_arr = []
    for i in range(len(json_arr)):
        j_vocabs_arr = []
        for j in range(len(json_arr[i])):
            value = json_arr[i][j]
            j_vocabs_arr.append(vocab[value])
        vocabs_arr.append(j_vocabs_arr)
            
    return vocabs_arr

In [7]:
tok_time = tokenize_arr(time_of_act, time_vocab[0])
tok_place = tokenize_arr(place_of_act, place_vocab[0])
tok_quality = tokenize_arr(qualities, qua_vocab[0])
tok_based_on = tokenize_arr(based_on, based_vocab[0])
tok_audience = tokenize_arr(audience, audi_vocab[0])
tok_mood = tokenize_arr(mood, mood_vocab[0])
tok_subgenre = tokenize_arr(subgenre, subgenr_vocab[0])
tok_about = tokenize_arr(about, about_vocab[0])
tok_theme = tokenize_arr(theme, theme_vocab[0])

In [8]:
trailers_meta_preprocessed = []
in_sess_preprocessed = []
out_sess_preprocessed = []
for i in range(len(tok_time)):
    trailers_meta_preprocessed.append([
#         content_id[i],
        tok_time[i],
        tok_place[i],
        tok_quality[i],
        tok_based_on[i],
        tok_audience[i],
        tok_mood[i],
        tok_subgenre[i],
        tok_about[i],
        tok_theme[i]
    ])

    
for i in range(len(in_sess)):
    in_sess_preprocessed_j = []
    for j in range(len(in_sess[i])):
        ind = content_id.tolist().index(in_sess[i][j])
        if ind:
            in_sess_preprocessed_j.append([
#                 content_id[ind],
                tok_time[ind][0],
                tok_place[ind][0],
                tok_quality[ind][0],
                tok_based_on[ind][0],
                tok_audience[ind][0],
                tok_mood[ind][0],
                tok_subgenre[ind][0],
                tok_about[ind][0],
                tok_theme[ind][0]
            ])
    in_sess_preprocessed.append(in_sess_preprocessed_j)
    
for i in range(len(out_sess)):
    out_sess_preprocessed_j = []
    for j in range(len(out_sess[i])):
        ind = content_id.tolist().index(out_sess[i][j])
        if ind:
            out_sess_preprocessed_j.append([
#                 content_id[ind],
                tok_time[ind][0],
                tok_place[ind][0],
                tok_quality[ind][0],
                tok_based_on[ind][0],
                tok_audience[ind][0],
                tok_mood[ind][0],
                tok_subgenre[ind][0],
                tok_about[ind][0],
                tok_theme[ind][0]
            ])
    out_sess_preprocessed.append(out_sess_preprocessed_j)

In [9]:
# попробовать с 3d массивом, надо дополнить недостающее нулями
x = []
for i in range(len(in_sess_preprocessed)):
    for j in range(len(in_sess_preprocessed[i])):
        x.append(in_sess_preprocessed[i][j])
        
y = []
for i in range(len(out_sess_preprocessed)):
    for j in range(len(out_sess_preprocessed[i])):
        y.append(out_sess_preprocessed[i][j])

In [10]:
max_id = 500000
model = keras.models.Sequential([
    keras.layers.GRU(249, return_sequences=True,
                     #dropout=0.2, recurrent_dropout=0.2,
                     dropout=0.2),
    keras.layers.GRU(128, return_sequences=True,
                     #dropout=0.2, recurrent_dropout=0.2),
                     dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id, activation="softmax"))
])

In [11]:
print(np.array(x).shape)
print(np.array(y[:-2]).shape)
x = np.array(x).astype(np.float)
y = np.array(y).astype(np.float)
x_train = np.reshape(x, (x.shape[0], x.shape[1], 1))
y_train = np.reshape(y[:-2], (y[:-2].shape[0], y[:-2].shape[1], 1))
# model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
# history = model.fit(np.asarray(x_train)[:200], np.asarray(y_train)[:200], epochs=20)

(249, 9)
(249, 9)


In [12]:
model.evaluate(np.asarray(x_train)[200:], np.asarray(y_train)[200:])

RuntimeError: You must compile your model before training/testing. Use `model.compile(optimizer, loss)`.

In [13]:
model.predict(x) # вывод

ValueError: in user code:

    /Users/nutochkina/PycharmProjects/diploma/venv/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1478 predict_function  *
        return step_function(self, iterator)
    /Users/nutochkina/PycharmProjects/diploma/venv/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1468 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /Users/nutochkina/PycharmProjects/diploma/venv/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /Users/nutochkina/PycharmProjects/diploma/venv/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /Users/nutochkina/PycharmProjects/diploma/venv/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:3417 _c

In [14]:
def pad(array, max_len, max_h):
    new_arr = []
    for i in range(max_len):
        new_arr_j = []
        if (i < np.array(array).shape[0]):
            for j in range(max_h):
                if j < np.array(array).shape[1]:
                    new_arr_j.append(array[i][j])
                else:
                    new_arr_j.append(0)
        else:
            for j in range(max_h):
                 new_arr_j.append(0)
        new_arr.append(new_arr_j)
    return new_arr

In [15]:
info_array = []
min_len = 100
max_len = 0
del_array_id = []
info_folder = "/Volumes/Seagate/natasha-diploma/videoinfo"
for i in range(0, len(trailers_meta)):
    name = trailers_meta.iloc[i]['trailers_name']
    content_id = trailers_meta.iloc[i]['content_id']
    try:
        info_csv = info_folder + '/' + name + '.csv'
        info_df =  pd.read_csv(info_csv, index_col=None, header=0)
        info_df = info_df.iloc[: , 3:]
        np_info_df = info_df.to_numpy()
        new_arr = []
        for i in range(len(np_info_df)):
            new_arr_j = []
            for j in range(len(np_info_df[i])):
                if isinstance(np_info_df[i][j], int) or isinstance(np_info_df[i][j], float):
                    new_arr_j.append(np_info_df[i][j])
                else:
                    nums = [int(s) for s in np_info_df[i][j].split() if s.isdigit()]
                    new_arr_j.append(float(np.mean(nums)))
            new_arr.append(new_arr_j)
            
        info_array.append(preprocessing.normalize(new_arr))        
#         print(info_array[i])
        if len(info_array[i]) < min_len:
            min_len = len(info_array[i])
            min_name = name
        if len(info_array[i]) > max_len:
            max_len = len(info_array[i])
            max_name = name
    except:
        del_array_id.append(content_id)
        
print(min_len, max_len)
padded = []
for i in range (len(info_array)):
    print(i)
    padded.append(pad(info_array[i], max_len, 23))
print(padded)

/Users/nutochkina/PycharmProjects/diploma/venv/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/nutochkina/PycharmProjects/diploma/venv/lib/python3.8/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/Users/nutochkina/PycharmProjects/diploma/venv/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/nutochkina/PycharmProjects/diploma/venv/lib/python3.8/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


6 396
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [16]:
shape = np.array(padded).shape
print(shape)

(909, 396, 23)


In [17]:
#encoder
encoder_input = Input(shape=(shape[1], shape[2]))
encoder_LSTM1 = LSTM(shape[0], return_sequences=True, dropout=0.2)
encoder_outputs = encoder_LSTM1(encoder_input)
encoder_LSTM2 = LSTM(249, return_state=True)
encoder_outputs, encoder_h, encoder_c = encoder_LSTM2(encoder_outputs)
encoder_states = [encoder_h, encoder_c]


In [18]:
#decoder
decoder_input = Input(shape=(9, 1))
decoder_LSTM1 = LSTM(249, return_sequences=True, dropout=0.2)
decoder_output = decoder_LSTM1(decoder_input, initial_state=encoder_states)
decoder_LSTM2 = LSTM(128, return_sequences=True, return_state=True)
decoder_output, _, _ = decoder_LSTM2(decoder_output)
decoder_dense = Dense(912, activation='softmax')
decoder_output = decoder_dense(decoder_output)

In [39]:
model = Model(inputs=[encoder_input, decoder_input], outputs=[decoder_output])
model.compile(optimizer='adam', loss="sparse_categorical_crossentropy")


In [69]:
print(x_train.shape)
x = np.asarray(x_train)[:200].tolist()
y = np.asarray(y_train)[:200].tolist()

for i in range(709):
    x.append(np.asarray(np.zeros((9, 1))))
    y.append(np.asarray(np.zeros((9, 1))))
    


(249, 9, 1)


In [71]:
model.fit(x=[np.asarray(padded), np.asarray(x)], y=np.asarray(y), epochs=20)

Epoch 1/20
18/29 [=================>............] - ETA: 49s - loss: 0.6053

KeyboardInterrupt: 

In [ ]:
print(padded[888])

In [45]:
print(x_train.shape)

(249, 9, 1)


In [59]:
x = np.asarray(x_train)[200:].tolist()
y = np.asarray(y_train)[200:].tolist()

for i in range(909-49):
    x.append(np.asarray(np.zeros((9, 1))))
    y.append(np.asarray(np.zeros((9, 1))))
    
print(len(x))
print(len(y))

909
909


In [61]:
model.evaluate([np.asarray(padded), np.asarray(x)], np.asarray(y))

29/29 [==============================] - 25s 811ms/step - loss: 0.1766


0.17655621469020844

In [63]:
res = model.predict([np.asarray(padded), np.asarray(x)])

In [68]:
print(res)

[[0.7964889561292661, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.359775832257999e-05, 0.016662996671261403, 0.00012252277522148975, 2.457878069831108e-11, 5.7847643877053234e-11, -0.003590966504990844, 0.5410445232518892, -0.0023109506792094586, 3.546966767222693e-05, 2.2142612341759125e-06, 0.26940723492152124, 0], [0.796488860475226, 0.0, 0.0, 0.0, 0.0004900910420286535, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.359775428766991e-05, 0.01666299467012512, 0.00012252276050716336, 2.4578777746531704e-11, 5.7847636929862314e-11, -0.0035909660737350794, 0.5410444582753517, -0.0023109504016767105, 3.5469663412510604e-05, 2.214260968255047e-06, 0.2694072025671612, 0], [0.796488860475226, 0.0, 0.0, 0.0, 0.0004900910420286535, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.359775428766991e-05, 0.01666299467012512, 0.00012252276050716336, 2.4578777746531704e-11, 5.7847636929862314e-11, -0.0035909660737350794, 0.5410444582753517, -0.0023109504016767105, 3.5469663412510604e-05, 2.214260968255047e-06, 0.269407202

[[1.0], [45.0], [187.0], [221.0], [247.0], [275.0], [314.0], [374.0], [888.0]]
